In [38]:
import numpy as np

In [39]:
import torch
from torch import nn

In [40]:
from sklearn.cluster import MiniBatchKMeans

In [41]:
def load_treshold():
    return 0.7

In [42]:
treshold = load_treshold()

In [43]:
kmeans = MiniBatchKMeans(n_clusters=200, random_state=0, compute_labels=True)

In [44]:
def read_string_array(arr, dtype=None):
    return np.fromstring(arr[1:-1], dtype=None, sep=' ')

In [45]:
def read_string_ndarray(arr, dtype=None):
    return np.array([read_string_array(target, dtype=None) for target in arr])

In [46]:
import pandas as pd

In [47]:
train_file = '../data/train.csv'
validation_file = '../data/validation.csv'

In [48]:
def read_chunk(chunk):
    X = chunk['image'].values
    y = chunk['label'].to_numpy(dtype=np.int)
    
    return X, y

In [49]:
def to_tensor(X, y, device=torch.device('cpu')):
    return torch.tensor(X, device=device), torch.tensor(y, device=device)

In [50]:
from sklearn.preprocessing import normalize

In [51]:
def fit(kmeans, train_file, chunksize=100000):
    count = 0
    train_df = pd.read_csv(train_file,
                           chunksize=chunksize,
                           header=0,
                           converters={'image': read_string_array})
    for chunk in train_df:
        print(count + len(chunk))
        X, y = read_chunk(chunk)
        X = np.array(X.tolist(), dtype=np.float)
        X = normalize(X, axis=1)
        kmeans.partial_fit(X, y)
        count += len(chunk)

In [52]:
device = torch.device('cuda:3')

In [53]:
# fit(kmeans, train_file)

In [54]:
import joblib

In [55]:
kmeans_filename = 'kmeans.joblib'

In [56]:
# joblib.dump(kmeans, kmeans_filename)

In [57]:
kmeans = joblib.load(kmeans_filename)

In [58]:
def predict_cluster(kmeans, file, chunksize=100000):
    count = 0
    df = pd.read_csv(file,
                     chunksize=chunksize,
                     header=0,
                     converters={'image': read_string_array})
    
    centers = torch.tensor(kmeans.cluster_centers_).to(device)
    result = []
    
    for chunk in df:
        print(count + len(chunk))
        X, y = read_chunk(chunk)
        X = np.array(X.tolist(), dtype=np.float)
        
        clusters = kmeans.predict(X)
        result.append(clusters)
        
        count += len(chunk)
    return np.concatenate(result)

In [59]:
preds = predict_cluster(kmeans, train_file, 100000)

In [60]:
def get_buckets(cluster_predictions):
    buckets = dict()
    
    for i, cluster in enumerate(cluster_predictions):
        if buckets.get(cluster, None) is None:
            buckets.update({cluster: []})
        buckets[cluster].append(i)
    return buckets

In [61]:
# buckets = get_buckets(cluster_predictions=preds)
# buckets

In [62]:
def save_buckets(buckets, file='../data/buckets.npy'):
    np.save(file, [buckets])

In [63]:
def load_buckets(file='../data/buckets.npy'):
    return np.load(file, allow_pickle=True)[0]

In [64]:
# save_buckets(buckets)

In [65]:
buckets = load_buckets()
# buckets

In [66]:
def chunk_iterator(df, chunksize=1, limit=None):
    length = len(df)
    i = 0
    
    while True:
        mn = i * chunksize
        mx = mn + min(chunksize, length)

        if limit is not None:
            mx = min(mx, limit)
        
        yield df[mn:mx]
        i += 1
        
        if mx >= length or (limit is not None and mx >= limit):
            raise StopIteration
        

In [67]:
def predict_order(kmeans, file, chunksize=100000):
    similiarity = torch.nn.CosineSimilarity(dim=1)
    
    count = 0
    df = pd.read_csv(file,
                     chunksize=chunksize,
                     header=0,
                     converters={'image': read_string_array})
    
    centers = torch.tensor(kmeans.cluster_centers_).to(device)
    result = []
    
    for chunk in df:
        print(count + len(chunk))
        X, y = read_chunk(chunk)
        X, y = to_tensor(X, y, device)
        
        sim = torch.stack([similiarity(center, X) for center in centers])
        sim = torch.transpose(sim, 1, 0)
        clusters_sorted = torch.argsort(-sim, dim=1)
        result.append(clusters_sorted)
        count += len(chunk)
    return torch.cat(result, dim=0)

In [68]:
# buckets_order = predict_order(kmeans, validation_file, 50000)
# buckets_order

In [69]:
def find(embedding, database_path, buckets, buckets_order, treshold, chunksize=100000):
    similiarity = torch.nn.CosineSimilarity(dim=1)
    
    for i in buckets_order:
        bucket = np.array(buckets[i])
        min_index = np.min(bucket)
        print(i)
        
        df = pd.read_csv(database_path,
                 chunksize=chunksize,
                 header=0)
        for i, chunk in enumerate(df):
            mn = i * chunksize
            mx = mn + len(chunk)
            
            if min_index > mx:
                continue
            
            condition = np.logical_and(
                                bucket <= mx,
                                bucket >= mn
                        )
            indexes = bucket[condition] - mn

            if not len(indexes):
                continue
                
            X, y = read_chunk(chunk)
            X = read_string_ndarray(X[indexes])
            y = y[indexes]
#             print(y)
            X = torch.tensor(X, device=device)
#             print(X, np.shape(X), np.shape(torch.stack([embedding[0]] * 3)))
            sim = similiarity(embedding, X)
            index = torch.argmax(sim)
            
            if sim[index] > treshold:
                return y[index.cpu()]
    

In [70]:
# df = pd.read_csv(validation_file,
#                  chunksize=1,
#                  header=0,
#                  converters={'image': read_string_array})
# imgs, label = read_chunk(df.get_chunk())
# imgs = np.array(imgs.tolist(), dtype=np.float)
# imgs = torch.tensor(imgs, device=device)
# print(label)
# find(imgs, train_file, buckets, buckets_order[0].cpu().numpy(), treshold)
# # print(label)

In [71]:
def collect_by_indexes(df, indexes, chunksize=100000):
    db = chunk_iterator(df, chunksize=chunksize)
    X_result = []
    y_result = []
    
    for chunk in db:
        X, y = read_chunk(chunk)
        X = np.take(X, indexes, mode='clip')
        X = read_string_ndarray(X)
        X = np.array(X.tolist(), dtype=np.float)
        y = np.take(y, indexes, mode='clip')
        
        X_result.append(X)
        y_result.append(y)
    return np.concatenate(X_result), np.concatenate(y_result)

In [72]:
def find_straight_forward(query, database_file, chunksize=100000, device=device):
    similiarity = torch.nn.CosineSimilarity(dim=1)
    
    count = 0
    df = pd.read_csv(database_file,
                     chunksize=chunksize,
                     header=0,
                     converters={'image': read_string_array})
    
    Q = torch.tensor([query], device=device)
    
    result = None
    best = -1
    
    for chunk in df:
        print(count + len(chunk))
        X, y = read_chunk(chunk)
        X = np.array(X.tolist(), dtype=np.float)
        
        X, y = to_tensor(X, y, device)
        
        print(np.shape(X), np.shape(Q))
        sim = similiarity(Q, X)
        
        idx = torch.argmax(sim).cpu().numpy()
        
        if sim[idx] > best:
            best = sim[idx]
            result = y[idx]
    return result
        
        

In [73]:
def find_by_treshold(query, df, treshold, chunksize=100000, device=device, device_ids=[1, 2, 3]):
    similiarity = torch.nn.CosineSimilarity(dim=1)
    
    count = 0
    
    Q = torch.tensor([query], device=device)
    treshold = torch.tensor(treshold, device=device)
    
    for chunk in chunk_iterator(df, chunksize=chunksize):
        print(count + len(chunk))
        X, y = read_chunk(chunk)
        X = read_string_ndarray(X)
        X = np.array(X.tolist(), dtype=np.float)
        
        X, y = to_tensor(X, y, device)
        
        sim = similiarity(Q, X)
        
        idx = torch.argmax(sim).cpu().numpy()
        
        if sim[idx] > treshold:
            return y[idx]
        

In [74]:
def find_in_clusters(kmeans, clusters, query, df, treshold, chunksize=100000, device=device):
    similiarity = torch.nn.CosineSimilarity(dim=1)
    similiarity = similiarity.to(device)
    
    count = 0
    
    Q = torch.tensor([query], device=device)
    treshold = torch.tensor(treshold, device=device)
    centers = torch.tensor(kmeans.cluster_centers_, device=device)
    
    sim = similiarity(Q, centers)
    clusters_sorted = torch.argsort(-sim).cpu().numpy()
#     print(clusters_sorted)
    
    for cluster_idx in clusters_sorted:
        print(cluster_idx)
        cluster = clusters.get(cluster_idx)
        if not cluster:
            continue
        cluster = np.array(clusters[cluster_idx])
        
        for indexes in chunk_iterator(cluster, chunksize=chunksize):
            X, y = collect_by_indexes(df, indexes, chunksize)
            
            X, y = to_tensor(X, y, device)

            sim = similiarity(Q, X)

            idx = torch.argmax(sim)

            if sim[idx] > treshold:
                return y[idx]
    return None

In [75]:
# from sklearn.metrics.pairwise import cosine_similarity

In [76]:
# def find_in_clusters(kmeans, clusters, query, df, treshold, chunksize=100000, device=device):
#     count = 0
    
#     centers = kmeans.cluster_centers_
    
#     sim = cosine_similarity([query], centers)[0]
#     clusters_sorted = np.argsort(-sim)
# #     print(clusters_sorted)
    
#     for cluster_idx in clusters_sorted:
#         cluster = np.array(clusters[cluster_idx])
        
#         for indexes in chunk_iterator(cluster, chunksize=chunksize):
#             X, y = collect_by_indexes(df, indexes, chunksize)
            
# #             X, y = to_tensor(X, y, device)

#             sim = cosine_similarity([query], X)[0]

#             idx = np.argmax(sim)

#             if sim[idx] > treshold:
#                 return y[idx]
#     return None

In [77]:
df = pd.read_csv(validation_file,
                 chunksize=1,
                 skiprows=20000,
                 converters={'image': read_string_array}, names=['image', 'label'])
imgs, label = read_chunk(df.get_chunk())
img = np.array(imgs.tolist()[0], dtype=np.float)

In [41]:
label

array([777])

In [78]:
import time

In [45]:
# start = time.time()
# label_hat = find_straight_forward(img, train_file, chunksize=50000)
# end = time.time()
# duration = end - start
# print('Duration:', duration)
# print('Pred:', label_hat, 'Real', label)

In [46]:
start = time.time()
label_hat = find_by_treshold(img, train_df, 0.7, chunksize=50000)
end = time.time()
duration = end - start
print('Duration:', duration)
print('Pred:', label_hat, 'Real', label)

50000
Duration: 8.177510738372803
Pred: tensor(777, device='cuda:1') Real [777]


In [47]:
start = time.time()
label_hat = find_in_clusters(kmeans, buckets, img, train_df, 0.7, chunksize=50000)
end = time.time()
duration = end - start
print('Duration:', duration)
print('Pred:', label_hat, 'Real', label)

155
Duration: 2.1036922931671143
Pred: tensor(777, device='cuda:1') Real [777]


/home/a.y.igoshin/virtual_environments/main/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: generator 'chunk_iterator' raised StopIteration
  


In [79]:
def timed(func, sample):
    timings = []
    labels = []
    labels_hat = []
    for item in chunk_iterator(sample, 1):
        item, label = read_chunk(item)
        item = np.array(item.tolist()[0], dtype=np.float)
        start = time.time()
        label_hat = func(item)
        end = time.time()
        duration = end - start
        timings.append(duration)
        if label_hat:
            label_hat = label_hat.cpu().numpy()
        labels.append(label)
        labels_hat.append()
        print('Duration:', duration)
        print('Pred:', label_hat, 'Real', label)
        
    return timings, labels, labels_hat

In [155]:
df_test = pd.read_csv(validation_file,
                 header=0,
                 converters={'image': read_string_array})


In [156]:
sample = df_test.sample(100, random_state=42)
sample.head()

,image,label
76678,"[0.122607708, -0.113119252, -0.241234466, -0.0...",4493
67269,"[-0.157144561, 0.147892162, -0.59451431, 0.255...",3910
9246,"[-0.144163221, -0.21046187, 0.0180296004, 0.01...",333
108186,"[0.112650268, -0.0320874378, 0.0571430661, 0.1...",6769
26504,"[0.377457768, -0.265060902, -0.13687636, 0.039...",985


In [159]:
timings, labels, labels_hat = timed(lambda img: find_in_clusters(kmeans, buckets, img, train_df, 0.7, chunksize=50000), sample)
print(timings)
print('Sum:', np.sum(timings), 'Mean:', np.mean(timings))
res = pd.DataFrame.from_records(list(zip(labels, labels_hat, timings)), columns=['label', 'label_hat', 'duration']) 
res.to_csv('clustering_results.csv', index=False)

141


/home/a.y.igoshin/virtual_environments/main/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: generator 'chunk_iterator' raised StopIteration
  
/home/a.y.igoshin/virtual_environments/main/lib/python3.6/site-packages/ipykernel_launcher.py:21: DeprecationWarning: generator 'chunk_iterator' raised StopIteration


178
190
143
118
52
132
41
157
36
89
78
131
133
33
74
73
Duration: 45.19784188270569
Pred: tensor(4493, device='cuda:1') Real [4493]
197
Duration: 1.3244633674621582
Pred: tensor(3910, device='cuda:1') Real [3910]
183
Duration: 1.504631519317627
Pred: tensor(333, device='cuda:1') Real [333]
181
69
177
87
128
30
15
55
41
4
106
112
102
7
169
129
193
70
188
43
64
10
140
28
183
54
2
189
192
56
9
144
136
71
89
53
113
115
59
139
49
172
150
131
111
24
109
38
93
22
79
163
108
138
44
100
167
195
62
141
135
194
107
3
126
29
197
117
184
174
165
124
76
118
152
35
94
104
27
52
156
66
85
199
149
190
11
42
153
20
123
39
176
40
105
18
170
143
157
92
78
120
14
67
36
6
45
155
60
8
103
37
198
88
158
81
122
77
58
173
48
75
110
178
146
125
134
116
17
0
99
13
16
32
187
147
68
142
95
19
186
132
148
74
171
5
127
12
160
180
185
25
33
130
182
73
196
46
57
84
168
161
162
65
175
34
82
133
166
96
90
154
179
86
21
191
26
137
101
47
121
83
151
145
63
72
91
61
1
50
119
97
159
98
164
114
23
31
80
51


AttributeError: 'NoneType' object has no attribute 'cpu'

In [ ]:
timings_by_treshold, labels, label_hat = timed(lambda img: find_by_treshold(img, train_df, 0.73, chunksize=50000), sample)
print(timings)
print('Sum:', np.sum(timings), 'Mean:', np.mean(timings))
res = pd.DataFrame.from_records(list(zip(labels, labels_hat, timings_by_treshold)), columns=['label', 'label_hat', 'duration']) 
res.to_csv('by_treshold_results.csv', index=False)

In [80]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(range(0, 10), timings)
plt.plot(range(0, 10), timings_by_treshold)
plt.show()

In [81]:
from sklearn.utils import shuffle

In [99]:
df_train = pd.read_csv(train_file,
                 header=0,
                 converters={'image': read_string_array})
df_train = shuffle(df_train)

In [45]:
# def pairs_iterator(df, chunksize=50000):
#     unique_labels = df["label"].unique()
    
#     for label in unique_labels:
#         positive = df[df["label"] == label]
#         negative = df[df["label"] == label]

#         size = int(chunksize / 4)
#         positive_1 = positive.sample(size)
#         positive_2 = positive.sample(size)
#         positive_3 = positive.sample(size)
#         negative_1 = negative.sample(size)
        
#         x1_1, y1_1 = read_chunk(positive_1)
#         x1_1 = np.array(x1_1.tolist(), dtype=np.float)
        
#         x1_2, y1_2 = read_chunk(positive_2)
#         x1_2 = np.array(x1_2.tolist(), dtype=np.float)
        
#         x2_1, y2_1 = read_chunk(positive_3)
#         x2_1 = np.array(x2_1.tolist(), dtype=np.float)
        
#         x2_2, y2_2 = read_chunk(negative_1)
#         x2_2 = np.array(x2_2.tolist(), dtype=np.float)
        
#         x1 = np.dstack((x1_1, x1_2))
#         x1 = x1.reshape(-1, x1.shape[1])
#         y1 = (y1_1 == y1_2).reshape(-1, 1).astype(np.int)
        
#         x2 = np.dstack((x2_1, x2_2))
#         x2 = x2.reshape(-1, x2.shape[1])
#         y2 = (y2_1 == y2_2).reshape(-1, 1).astype(np.int)
        
#         X = np.concatenate([x1, x2])
#         Y = np.concatenate([y1, y2])
        
#     for chunk_1 in chunk_iterator(df, chunksize):
#         for item in chunk_iterator(chunk_1, 1):
            

            
            
#             for chunk_2 in chunk_iterator(df, chunksize):
#                 x2, y2 = read_chunk(chunk_2)
#                 x2 = np.array(x2.tolist(), dtype=np.float)
#                 x1 = x1[:len(x2)]
                

#                 X = np.dstack((x1, x2))
#                 X = X.reshape(-1, X.shape[1])

#                 Y = (y1 == y2).reshape(-1, 1).astype(np.int)
#                 Y = Y + (Y - 1)

#                 yield X, Y
#     raise StopIteration

In [82]:
from sklearn.preprocessing import normalize as _normalize

In [83]:
def pairs_iterator(df, chunksize=50000, normalize=False):
    unique_labels = df["label"].unique()
    
    for label in unique_labels:
        positive = df[df["label"] == label]
        negative = df[df["label"] != label]

        size = int(chunksize / 4)
        positive_1 = positive.sample(size, replace=True)
        positive_2 = positive.sample(size, replace=True)
        positive_3 = positive.sample(size, replace=True)
        negative_1 = negative.sample(size, replace=True)
        
        x1_1, y1_1 = read_chunk(positive_1)
        x1_1 = np.array(x1_1.tolist(), dtype=np.float)
        
        x1_2, y1_2 = read_chunk(positive_2)
        x1_2 = np.array(x1_2.tolist(), dtype=np.float)
        
        x2_1, y2_1 = read_chunk(positive_3)
        x2_1 = np.array(x2_1.tolist(), dtype=np.float)
        
        x2_2, y2_2 = read_chunk(negative_1)
        x2_2 = np.array(x2_2.tolist(), dtype=np.float)
        
        x1 = np.array(list(zip(x1_1, x1_2)))
        x1 = np.concatenate(x1, axis=0)
        y1 = (y1_1 == y1_2).reshape(-1, 1).astype(np.int)
        
        x2 = np.array(list(zip(x2_1, x2_2)))
        x2 = np.concatenate(x2, axis=0)
        y2 = -(y2_1 != y2_2).reshape(-1, 1).astype(np.int)
        
        X = np.concatenate([x1, x2])
        Y = np.concatenate([y1, y2])
        
        if normalize:
            X = _normalize(X, axis=1)
        
        yield X, Y
    raise StopIteration

In [84]:
pairs_iterator(df_train, 10)

NameError: name 'df_train' is not defined

In [85]:
from torch.nn import Parameter

In [86]:
class Model(nn.Module):
    def __init__(self, buckets):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(512, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(256, 128)
    def forward(self, X):
        X = self.fc1(X)
        X = self.bn1(X)
        X = self.relu1(X)
        X = self.fc2(X)
        return X

class Head(nn.Module):
    def __init__(self, buckets, hidden=512):
        super(Head, self).__init__()
        self.embeddings = Parameter(torch.rand((buckets, hidden)))
    def forward(self, X):
        X = nn.functional.gumbel_softmax(X)
        X = X.mm(self.embeddings)
        return X

In [87]:
net = Model(buckets=128)
net = net.to(device)

head = Head(buckets=128)
head = head.to(device)

In [88]:
# net.load_state_dict(torch.load('model.torch'))
# head.load_state_dict(torch.load('head.torch'))

In [89]:
# df_train.t

In [90]:
head.embeddings

Parameter containing:
tensor([[0.3067, 0.4365, 0.8953,  ..., 0.3210, 0.4284, 0.5551],
        [0.3705, 0.2401, 0.8905,  ..., 0.8575, 0.6099, 0.1148],
        [0.7211, 0.4454, 0.6971,  ..., 0.6820, 0.8625, 0.5353],
        ...,
        [0.8806, 0.7916, 0.4146,  ..., 0.3049, 0.6033, 0.2776],
        [0.5607, 0.6085, 0.5082,  ..., 0.7436, 0.1212, 0.2087],
        [0.7053, 0.7387, 0.7267,  ..., 0.2930, 0.5021, 0.5407]],
       device='cuda:3', requires_grad=True)

In [51]:
def balanced_accuracy(y, y_hat):
    tp = torch.sum((y + 1) & (y_hat + 1)).type(torch.float32) / 2
    tn = - torch.sum((y - 1) & (y_hat - 1)).type(torch.float32) / 2
    p_count = torch.sum((y + 1)).type(torch.float32) / 2
    n_count = - torch.sum((y - 1)).type(torch.float32) / 2
    
    if n_count == 0:
        return tp / p_count
    
    if p_count == 0:
        return tn / n_count
    
    return (tp / p_count + tn / n_count) / 2

In [98]:
def recall_at_k(x1, x2, y, k=5):
    y = y > 0
    is_same = y.reshape(len(y))
    y = y[is_same]
    count = torch.sum(is_same)
    
    if not count:
        return 1.0
    
    x1 = x1.detach()[is_same, :].argmax(axis=1).reshape(-1, 1)
    x2 = -x2.detach()[is_same, :]
    x2 = x2.argsort(axis=1)[:, :k]
    y_hat = x1 == x2
    y_hat = torch.sum(y_hat, dim=1).reshape(-1, 1) > 0
    y = y > 0
    recall = torch.sum(y == y_hat).type(torch.float) / count
    return recall

In [57]:
recall_at_k(torch.tensor([[0.9, 0.1, 0.5, 0.6, 0.7],[0.1, 0.5 , 0.2, 0.3, 0.1]]), 
            torch.tensor([[1.0, 0.0, 0.2, 0.3, 0.4], [0.2, 1 , 0.3, 0.5, 0.6]]),
            torch.tensor([[1],[-1]]), k=2)

tensor(1.)

In [58]:
# kmeans = MiniBatchKMeans(n_clusters=200, random_state=0, compute_labels=True)

In [49]:
from torch.autograd import Variable

In [60]:
x = torch.tensor([2.0], device=device, requires_grad=True)
y = torch.tensor([1.0], device=device, requires_grad=True)
z = (x - y)**2
z.backward()
print(x.grad)
print(y.grad)
z = torch.tensor([1], device=device)

tensor([2.], device='cuda:0')
tensor([-2.], device='cuda:0')


In [169]:
def cosine_loss(x1, x2, y, margin=0, reduction='mean'):
    sim = torch.nn.functional.cosine_similarity(x1, x2, dim=1)
    positive = (y > 0).view(-1)
    positive_loss = sim[positive]
    positive_loss = 1 - positive_loss
    negative = (y < 0).view(-1)
    negative_loss = sim[negative]
    negative_loss = torch.max(torch.zeros(negative_loss.shape, device=device), negative_loss - margin)
    
    if reduction == 'sum':
        return torch.sum(positive_loss) + torch.sum(negative_loss)
    
    return torch.mean(positive_loss) + torch.mean(negative_loss)

In [55]:
criterion = nn.CosineEmbeddingLoss(margin=-1, reduction='sum')
def get_optimizer(lr):
    return torch.optim.SGD([{'params': net.parameters()}, {'params': head.parameters()}],
                    lr=lr, nesterov=True, momentum=0.9, weight_decay=0.1)

In [80]:
net.train()
head.train()
lr = 0.1
for epoch in range(5):
    train_loss = 0
    train_recall = 0.
    count = 0
    optimizer = get_optimizer(lr)
    for X, y in pairs_iterator(df_train, chunksize=20000, normalize=False):
        X = torch.tensor(X, device=device, dtype=torch.float, requires_grad=True)
        y = torch.tensor(y, device=device, dtype=torch.int)

        optimizer.zero_grad()
        
        p = net(X)
        thetas = head(p)
        
        
        x1, x2 = thetas[0::2], thetas[1::2]
        recall = recall_at_k(x1, x2, y)
        loss = cosine_loss(x1, x2, y, -1)
#         loss = criterion(x1, x2, y)
        loss.backward()
#         print(head.embeddings.grad)
        optimizer.step()
        train_loss += loss.item()
        train_recall += recall.cpu().detach().numpy()
        count += 1
        print('Epoch:', epoch, 'Loss:', train_loss // count, 'Recall:', train_recall / count)
    lr = max(lr / 10, 0.001)
        
        

Epoch: 0 Loss: 9998.0 Recall: 0.02160000056028366
Epoch: 0 Loss: 9996.0 Recall: 0.0213000001385808
Epoch: 0 Loss: 10000.0 Recall: 0.022200000161925953
Epoch: 0 Loss: 9999.0 Recall: 0.022450000047683716
Epoch: 0 Loss: 10000.0 Recall: 0.023600000143051147
Epoch: 0 Loss: 10000.0 Recall: 0.023933333344757557
Epoch: 0 Loss: 10000.0 Recall: 0.02374285725610597
Epoch: 0 Loss: 10000.0 Recall: 0.02445000014267862
Epoch: 0 Loss: 9994.0 Recall: 0.07186666772597367
Epoch: 0 Loss: 9934.0 Recall: 0.15489999931305648
Epoch: 0 Loss: 9732.0 Recall: 0.203709090128541
Epoch: 0 Loss: 9530.0 Recall: 0.2374999993480742
Epoch: 0 Loss: 9561.0 Recall: 0.28978461700563246
Epoch: 0 Loss: 9577.0 Recall: 0.3323428565636277
Epoch: 0 Loss: 9560.0 Recall: 0.3759866653631131
Epoch: 0 Loss: 9542.0 Recall: 0.41092499892693013
Epoch: 0 Loss: 9490.0 Recall: 0.41592941027792063
Epoch: 0 Loss: 9557.0 Recall: 0.42232222182469237
Epoch: 0 Loss: 9494.0 Recall: 0.44932631589472294
Epoch: 0 Loss: 9526.0 Recall: 0.456640000548213

Epoch: 0 Loss: 8764.0 Recall: 0.6837600011926364
Epoch: 0 Loss: 8762.0 Recall: 0.6835567262677247
Epoch: 0 Loss: 8761.0 Recall: 0.6829395361656193
Epoch: 0 Loss: 8769.0 Recall: 0.6810832381829878
Epoch: 0 Loss: 8765.0 Recall: 0.6809977022288688
Epoch: 0 Loss: 8757.0 Recall: 0.6810045725852251
Epoch: 0 Loss: 8762.0 Recall: 0.6799727283651009
Epoch: 0 Loss: 8753.0 Recall: 0.6810000011707935
Epoch: 0 Loss: 8757.0 Recall: 0.6803000011701095
Epoch: 0 Loss: 8765.0 Recall: 0.6786000010916474
Epoch: 0 Loss: 8772.0 Recall: 0.6772777789065407
Epoch: 0 Loss: 8766.0 Recall: 0.6779922661757436
Epoch: 0 Loss: 8757.0 Recall: 0.6782186823784486
Epoch: 0 Loss: 8756.0 Recall: 0.6770502742420618
Epoch: 0 Loss: 8762.0 Recall: 0.6760554357290106
Epoch: 0 Loss: 8767.0 Recall: 0.675284325260971
Epoch: 0 Loss: 8773.0 Recall: 0.6740924739929778
Epoch: 0 Loss: 8766.0 Recall: 0.6755497335969286
Epoch: 0 Loss: 8765.0 Recall: 0.675086171217659
Epoch: 0 Loss: 8766.0 Recall: 0.6750444455969112
Epoch: 0 Loss: 8752.0 

Epoch: 0 Loss: 8686.0 Recall: 0.7085751481078697
Epoch: 0 Loss: 8689.0 Recall: 0.7089292036869445
Epoch: 0 Loss: 8690.0 Recall: 0.7097852942643359
Epoch: 0 Loss: 8690.0 Recall: 0.7106363637826223
Epoch: 0 Loss: 8688.0 Recall: 0.7114824562861818
Epoch: 0 Loss: 8690.0 Recall: 0.7119183674965152
Epoch: 0 Loss: 8691.0 Recall: 0.7124604652906487
Epoch: 0 Loss: 8692.0 Recall: 0.7129176812705355
Epoch: 0 Loss: 8690.0 Recall: 0.7137473989547246
Epoch: 0 Loss: 8688.0 Recall: 0.7145723344044228
Epoch: 0 Loss: 8676.0 Recall: 0.7148195403178448
Epoch: 0 Loss: 8681.0 Recall: 0.7142148997293075
Epoch: 0 Loss: 8678.0 Recall: 0.7150314285872238
Epoch: 0 Loss: 8674.0 Recall: 0.7156860399566739
Epoch: 0 Loss: 8678.0 Recall: 0.7151903409288604
Epoch: 0 Loss: 8680.0 Recall: 0.7149665722882022
Epoch: 0 Loss: 8674.0 Recall: 0.7150824858759672
Epoch: 0 Loss: 8659.0 Recall: 0.7158850704227955
Epoch: 0 Loss: 8663.0 Recall: 0.7153921349110145
Epoch: 0 Loss: 8664.0 Recall: 0.7152997199341613
Epoch: 0 Loss: 8665.

Epoch: 0 Loss: 8632.0 Recall: 0.7220078895393854
Epoch: 0 Loss: 8631.0 Recall: 0.7222669291764674
Epoch: 0 Loss: 8630.0 Recall: 0.7226565815691337
Epoch: 0 Loss: 8633.0 Recall: 0.7222270588627925
Epoch: 0 Loss: 8634.0 Recall: 0.7224336594831628
Epoch: 0 Loss: 8637.0 Recall: 0.7221382812676893
Epoch: 0 Loss: 8637.0 Recall: 0.7222553606001664
Epoch: 0 Loss: 8640.0 Recall: 0.7218225680632292
Epoch: 0 Loss: 8636.0 Recall: 0.721771650403304
Epoch: 0 Loss: 8635.0 Recall: 0.7221662790240185
Epoch: 0 Loss: 8635.0 Recall: 0.722425145034028
Epoch: 0 Loss: 8631.0 Recall: 0.7223123551299616
Epoch: 0 Loss: 8634.0 Recall: 0.7219025047032992
Epoch: 0 Loss: 8632.0 Recall: 0.7223134613918284
Epoch: 0 Loss: 8630.0 Recall: 0.7228464489899247
Epoch: 0 Loss: 8624.0 Recall: 0.7229984673239662
Epoch: 0 Loss: 8622.0 Recall: 0.7227694071921917
Epoch: 0 Loss: 8621.0 Recall: 0.7231080152115692
Epoch: 0 Loss: 8623.0 Recall: 0.722904380840205
Epoch: 0 Loss: 8620.0 Recall: 0.7232079846968612
Epoch: 0 Loss: 8617.0 R

Epoch: 0 Loss: 8606.0 Recall: 0.727971893440403
Epoch: 0 Loss: 8608.0 Recall: 0.7276531757217346
Epoch: 0 Loss: 8609.0 Recall: 0.7273737462896821
Epoch: 0 Loss: 8610.0 Recall: 0.727463622943708
Epoch: 0 Loss: 8611.0 Recall: 0.7275064705898437
Epoch: 0 Loss: 8613.0 Recall: 0.7274079294779533
Epoch: 0 Loss: 8611.0 Recall: 0.7278076245960209
Epoch: 0 Loss: 8613.0 Recall: 0.7275306002958235
Epoch: 0 Loss: 8612.0 Recall: 0.7277236842245343
Epoch: 0 Loss: 8608.0 Recall: 0.7280064233871054
Epoch: 0 Loss: 8608.0 Recall: 0.7276787172226403
Epoch: 0 Loss: 8609.0 Recall: 0.7277147015949827
Epoch: 0 Loss: 8604.0 Recall: 0.7281104651101064
Epoch: 0 Loss: 8599.0 Recall: 0.728367489115139
Epoch: 0 Loss: 8600.0 Recall: 0.7280944927852007
Epoch: 0 Loss: 8603.0 Recall: 0.7280173661367895
Epoch: 0 Loss: 8595.0 Recall: 0.7282546242682877
Epoch: 0 Loss: 8596.0 Recall: 0.7285968253595961
Epoch: 0 Loss: 8596.0 Recall: 0.7288965417920298
Epoch: 0 Loss: 8595.0 Recall: 0.7292866187103146
Epoch: 0 Loss: 8595.0 R

...

Epoch: 4 Loss: 8356.0 Recall: 0.7534113046347967
Epoch: 4 Loss: 8356.0 Recall: 0.7534416895978937
Epoch: 4 Loss: 8356.0 Recall: 0.7534898925911296
Epoch: 4 Loss: 8356.0 Recall: 0.753444331624575
Epoch: 4 Loss: 8357.0 Recall: 0.7534057065882392
Epoch: 4 Loss: 8357.0 Recall: 0.753364165804497
Epoch: 4 Loss: 8357.0 Recall: 0.7533635086125056
Epoch: 4 Loss: 8357.0 Recall: 0.7533808985550422
Epoch: 4 Loss: 8357.0 Recall: 0.7533773482893389
Epoch: 4 Loss: 8357.0 Recall: 0.7533914878155653
Epoch: 4 Loss: 8356.0 Recall: 0.7534000781942294
Epoch: 4 Loss: 8356.0 Recall: 0.7533964091718522
Epoch: 4 Loss: 8356.0 Recall: 0.7534445269456724
Epoch: 4 Loss: 8356.0 Recall: 0.7533937574328271
Epoch: 4 Loss: 8357.0 Recall: 0.7533597036518966
Epoch: 4 Loss: 8357.0 Recall: 0.753358697469307
Epoch: 4 Loss: 8357.0 Recall: 0.7533594074189303
Epoch: 4 Loss: 8357.0 Recall: 0.7533480313147253
Epoch: 4 Loss: 8357.0 Recall: 0.7533961022499592
Epoch: 4 Loss: 8357.0 Recall: 0.7533998052739855
Epoch: 4 Loss: 8356.0 R

Epoch: 4 Loss: 8356.0 Recall: 0.7528554441869721
Epoch: 4 Loss: 8355.0 Recall: 0.752856986128169
Epoch: 4 Loss: 8356.0 Recall: 0.752827484522402
Epoch: 4 Loss: 8356.0 Recall: 0.7528222560047833
Epoch: 4 Loss: 8356.0 Recall: 0.7528377295555433
Epoch: 4 Loss: 8356.0 Recall: 0.7528542944914096
Epoch: 4 Loss: 8355.0 Recall: 0.752875543912372
Epoch: 4 Loss: 8355.0 Recall: 0.7528813919573797
Epoch: 4 Loss: 8355.0 Recall: 0.7529281150823641
Epoch: 4 Loss: 8355.0 Recall: 0.7529076749863832
Epoch: 4 Loss: 8355.0 Recall: 0.7528661879941615
Epoch: 4 Loss: 8355.0 Recall: 0.7528887756354684
Epoch: 4 Loss: 8356.0 Recall: 0.7528602684037392
Epoch: 4 Loss: 8356.0 Recall: 0.7528465811626209
Epoch: 4 Loss: 8356.0 Recall: 0.7528018131584802
Epoch: 4 Loss: 8356.0 Recall: 0.7527881798832439
Epoch: 4 Loss: 8356.0 Recall: 0.7527826695548424
Epoch: 4 Loss: 8356.0 Recall: 0.7527811250717247
Epoch: 4 Loss: 8356.0 Recall: 0.7528277789450835
Epoch: 4 Loss: 8356.0 Recall: 0.7527785284226796
Epoch: 4 Loss: 8356.0 R

Epoch: 4 Loss: 8350.0 Recall: 0.752913578092505
Epoch: 4 Loss: 8350.0 Recall: 0.7529387636439251
Epoch: 4 Loss: 8350.0 Recall: 0.7529423332070045
Epoch: 4 Loss: 8350.0 Recall: 0.7529876399494936
Epoch: 4 Loss: 8350.0 Recall: 0.752961459595247
Epoch: 4 Loss: 8350.0 Recall: 0.7529452613468677
Epoch: 4 Loss: 8350.0 Recall: 0.7529905426406092
Epoch: 4 Loss: 8349.0 Recall: 0.7530358073386776
Epoch: 4 Loss: 8349.0 Recall: 0.7530090144131375
Epoch: 4 Loss: 8348.0 Recall: 0.7530357575848942
Epoch: 4 Loss: 8348.0 Recall: 0.7530720147676083
Epoch: 4 Loss: 8347.0 Recall: 0.7531024721860231
Epoch: 4 Loss: 8346.0 Recall: 0.7531476749556705
Epoch: 4 Loss: 8346.0 Recall: 0.7531745562116109
Epoch: 4 Loss: 8346.0 Recall: 0.7532197292430509
Epoch: 4 Loss: 8346.0 Recall: 0.7531818116354703
Epoch: 4 Loss: 8346.0 Recall: 0.7531384560179021
Epoch: 4 Loss: 8347.0 Recall: 0.7530956284228377
Epoch: 4 Loss: 8347.0 Recall: 0.7530827360294471
Epoch: 4 Loss: 8347.0 Recall: 0.7530432255639958
Epoch: 4 Loss: 8346.0 

Epoch: 4 Loss: 8345.0 Recall: 0.7526418655612113
Epoch: 4 Loss: 8345.0 Recall: 0.75259953741502
Epoch: 4 Loss: 8345.0 Recall: 0.7526435588496435
Epoch: 4 Loss: 8345.0 Recall: 0.7526574276389016
Epoch: 4 Loss: 8345.0 Recall: 0.7526273213748362
Epoch: 4 Loss: 8345.0 Recall: 0.7525821449004859
Epoch: 4 Loss: 8345.0 Recall: 0.7526261381179645
Epoch: 4 Loss: 8344.0 Recall: 0.7526701156934102
Epoch: 4 Loss: 8344.0 Recall: 0.752714077635164
Epoch: 4 Loss: 8343.0 Recall: 0.7527580239515608
Epoch: 4 Loss: 8343.0 Recall: 0.752716453588424
Epoch: 4 Loss: 8343.0 Recall: 0.7527170369120638
Epoch: 4 Loss: 8343.0 Recall: 0.7527043340684677
Epoch: 4 Loss: 8343.0 Recall: 0.7527202629763583
Epoch: 4 Loss: 8343.0 Recall: 0.7527204191799022
Epoch: 4 Loss: 8343.0 Recall: 0.7527428370014762
Epoch: 4 Loss: 8343.0 Recall: 0.7527132056835996
Epoch: 4 Loss: 8343.0 Recall: 0.7527147472592297
Epoch: 4 Loss: 8342.0 Recall: 0.7527586232799431
Epoch: 4 Loss: 8342.0 Recall: 0.752756501832329
Epoch: 4 Loss: 8342.0 Rec

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
net.requires_grad_(True)
head.requires_grad_(True)
net.train(True)
head.train(True)
# net.requiresGrad(True)
chunksize=20000

for epoch in range(20):  
    print('epoch', epoch)
#     df = pd.read_csv(train_file,
#                  chunksize=chunksize,
#                  header=0,
#                  converters={'image': read_string_array})
    valid = 0
    count = 0
    train_loss = 0.
    train_recall = 0.
    for X, y in pairs_iterator(df_train, chunksize):
        X = torch.tensor(X, device=device, dtype=torch.float, requires_grad=True)
        y = torch.tensor(y, device=device, dtype=torch.int)



        buckets = net(X)
        outp = head(buckets)
        
        
        optimizer.zero_grad()

        x1, x2 = outp[0::2], outp[1::2]
        loss_t = criterion(x1, x2, y)
        print(loss_t)
#         print(loss_t)
#         print(x1, x2)
#         print(buckets)
#         print(head.embeddings)

#         for param in net.parameters():
#             param[0][0].backward()
#             print(param.grad)
#         print(head.embeddings)
#         recall = recall_at_k(x1, x2, y)
#         print(loss_t.grad)
        backward(loss_t, create_graph=False)
#         for param in net.parameters():
#             print(param.grad)
#         torch.autograd.backward(loss_t)
#         print(loss_t.grad_fn(loss_t.data))
        print(loss_t.grad)
#         print(head.embeddings.grad)
       

#         optimizer.step()
#         print(outp.grad)
#         print(head.embeddings.grad)
#         print(outp.grad)
#         for param in net.parameters():
#             print(torch.max(param.grad))
#         outp[0][0].backward()
#         print(outp.grad)
#         print(head.embeddings.grad)
#         print(outp.grad)
#         for param in net.parameters():
#             print(torch.max(param.grad))

#         train_loss += loss_t.item()
#         count += len(y)
# #         print('Loss:', loss_t.item(), 'Acc:', balanced_accuracy(y, y_hat), 'Recall at 5:', recall)
#         print('Loss:', loss_t.item())

#         if i % 10 == 0:
#           torch.save(net.state_dict(), model_path)
#     train_loss /= (len(inp) // bs)

In [81]:
torch.save(net.state_dict(), 'model_3.torch')
torch.save(head.state_dict(), 'head_3.torch')

In [65]:
head.embeddings

Parameter containing:
tensor([[ 2.1114e-06,  2.8517e-06,  3.0691e-06,  ...,  7.1735e-06,
         -2.7967e-06, -2.1323e-06],
        [ 1.9572e-06,  1.2481e-05,  9.6055e-06,  ...,  2.0537e-05,
         -1.3621e-05, -1.4624e-05],
        [ 1.1783e-05,  1.7339e-05,  1.8107e-05,  ...,  4.2044e-05,
         -1.7205e-05, -1.3732e-05],
        ...,
        [ 1.3449e-06,  3.0871e-06,  2.8282e-06,  ...,  6.3632e-06,
         -3.2060e-06, -2.9918e-06],
        [ 4.8445e-06,  2.3684e-05,  1.8822e-05,  ...,  4.0657e-05,
         -2.5634e-05, -2.6929e-05],
        [ 7.2849e-06,  9.6702e-06,  1.0473e-05,  ...,  2.4513e-05,
         -9.4602e-06, -7.1398e-06]], device='cuda:0', requires_grad=True)

In [66]:
def predict_cluster_with_network(net, file, chunksize=100000):
    net.eval()
    head.eval()
    
    with torch.no_grad():
        count = 0
        df = pd.read_csv(file,
                         chunksize=chunksize,
                         header=0,
                         converters={'image': read_string_array})

        result = []

        for chunk in df:
            print(count + len(chunk))
            X, _ = read_chunk(chunk)
            X = np.array(X.tolist(), dtype=np.float)
            X = torch.tensor(X, device=device, dtype=torch.float)

            clusters = net(X).detach().argmax(axis=1)
            result.append(clusters)

            count += len(chunk)
        return torch.cat(result, 0).cpu().numpy()

In [67]:
preds = predict_cluster_with_network(net, train_file, 100000)

100000
200000
300000
343350


In [50]:
def get_buckets(cluster_predictions):
    buckets = dict()
    
    for i, cluster in enumerate(cluster_predictions):
        if buckets.get(cluster, None) is None:
            buckets.update({cluster: []})
        buckets[cluster].append(i)
    return buckets

In [69]:
clusters = get_buckets(cluster_predictions=preds)
clusters

{55: [0,
  2,
  3,
  5,
  8,
  9,
  10,
  11,
  14,
  15,
  17,
  18,
  20,
  25,
  28,
  29,
  31,
  32,
  35,
  36,
  38,
  39,
  40,
  41,
  42,
  43,
  45,
  47,
  48,
  51,
  58,
  59,
  60,
  62,
  65,
  67,
  68,
  73,
  76,
  77,
  78,
  83,
  84,
  87,
  88,
  89,
  91,
  93,
  95,
  98,
  99,
  100,
  113,
  114,
  115,
  116,
  118,
  121,
  125,
  126,
  127,
  129,
  135,
  139,
  140,
  141,
  143,
  144,
  145,
  146,
  150,
  151,
  153,
  155,
  156,
  160,
  161,
  162,
  163,
  167,
  170,
  172,
  173,
  174,
  175,
  178,
  179,
  180,
  181,
  186,
  188,
  189,
  190,
  193,
  194,
  196,
  198,
  199,
  201,
  204,
  205,
  206,
  207,
  210,
  213,
  214,
  215,
  221,
  222,
  224,
  226,
  227,
  230,
  231,
  233,
  235,
  237,
  238,
  239,
  241,
  242,
  247,
  248,
  249,
  252,
  253,
  256,
  259,
  261,
  262,
  263,
  264,
  266,
  267,
  269,
  272,
  276,
  279,
  282,
  283,
  290,
  293,
  295,
  296,
  297,
  298,
  300,
  301,
  302,
  306,
  3

In [70]:
len(clusters)

3

In [71]:
for i in clusters:
    print(len(clusters[i]))

170402
166672
6276


In [536]:
def find_with_network(net, clusters, query, df, treshold, chunksize=100000, device=device):
    net.eval()

    similiarity = torch.nn.CosineSimilarity(dim=1)
    similiarity = similiarity.to(device)
    similiarity.eval()

    with torch.no_grad():
        count = 0

        Q = torch.tensor([query], device=device, dtype=torch.float)
        treshold = torch.tensor(treshold, device=device)
        clusters_hat = net(Q)
        clusters_sorted = torch.argsort(-clusters_hat).cpu().numpy()[0]
        print(clusters_sorted)

        for cluster_idx in clusters_sorted:
            print(cluster_idx)
            cluster = clusters.get(cluster_idx)
            if not cluster:
                continue
            cluster = np.array(clusters[cluster_idx])

            for indexes in chunk_iterator(cluster, chunksize=chunksize):
                X, y = collect_by_indexes(df, indexes, chunksize)

                X, y = to_tensor(X, y, device)

                sim = similiarity(Q, X)

                idx = torch.argmax(sim)

                if sim[idx] > treshold:
                    return y[idx]
        return None

In [575]:
df = pd.read_csv(validation_file,
                 chunksize=1,
                 skiprows=20000,
                 converters={'image': read_string_array}, names=['image', 'label'])
imgs, label = read_chunk(df.get_chunk())
img = np.array(imgs.tolist()[0], dtype=np.float)

In [576]:
label

array([777])

In [577]:
print(label[0] in train_df['label'].values)
start = time.time()
label_hat = find_with_network(net, clusters, img, train_df, 0.7, chunksize=10000)
end = time.time()
duration = end - start
print('Duration:', duration)
print('Pred:', label_hat, 'Real', label)

True
[  4  29  60  66  85  59   1  12  20  36 117 127  72  27  58  62  89 113
   9 106 100  42  31  79  93  91 108  50  46 103   0  84  83  34 124  86
  57  73  32  40  11  47  65  99  74  10 122  97  19  70  68  82 101  17
 111  92 109  75  49 118  38  22 125 105 121  25 120   7  15  21  61  44
  16  13 112 102  77 116  71  14   6  48   2 107  41  63  33  18 126  98
 104 114   3  24  95  35 119  90   8  94  26  67 110  23  45   5  56  28
  43  55  78  96  54  76  88  51  30  87  64  69  81  80 115  53  39  52
  37 123]
4


/home/a.y.igoshin/virtual_environments/main/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: generator 'chunk_iterator' raised StopIteration
  
/home/a.y.igoshin/virtual_environments/main/lib/python3.6/site-packages/ipykernel_launcher.py:24: DeprecationWarning: generator 'chunk_iterator' raised StopIteration


29
60
66
Duration: 259.3990397453308
Pred: tensor(777, device='cuda:0') Real [777]


In [413]:
len(clusters[100])

45594

In [91]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(512, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(256, 128)

        
        self.fc3 = nn.Linear(512, 512)
        self.bn2 = nn.BatchNorm1d(512)
        self.relu2 = nn.ReLU()
        self.fc4 = nn.Linear(512, 512)
        
        self.embeddings = Parameter(torch.rand((128, 512)))

    def encode(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        return x

    def decode(self, z):
        z = self.fc3(z)
        z = self.bn2(z)
        z = self.relu2(z)
        z = self.fc4(z)
        return z
    
    def reparametrize(self, q_y):
        z = nn.functional.gumbel_softmax(q_y)
        z = torch.mm(z, self.embeddings)
        return z
    
    def forward(self, x):
        encoding = self.encode(x)
        z = self.reparametrize(encoding)
        return self.decode(z), z, encoding

In [92]:
from torch.nn import MSELoss, L1Loss, CosineEmbeddingLoss, KLDivLoss, BCELoss
import torch.nn.functional as F

In [93]:
class CosineLoss(nn.Module):
    def __init__(self, reduction='mean', margin=0):
        super(CosineLoss, self).__init__()
        self.margin = margin
        self.reduction = reduction
    def forward(self, x1, x2, y):
        sim = F.cosine_similarity(x1, x2, dim=1)
        y = y.view(-1)
        positive = y > 0
        positive_loss = 1 - positive * sim
        negative = y < 0
        negative_loss = negative * sim - self.margin
        negative_loss = torch.max(
            torch.zeros(negative_loss.shape, device=device),
            negative_loss
        )

        if self.reduction == 'sum':
            return torch.sum(positive_loss) + torch.sum(negative_loss)

        return torch.mean(positive_loss) + torch.mean(negative_loss)
        
        
class VaeLoss(nn.Module):
    def __init__(self, reduction='mean', margin=0):
        super(VaeLoss, self).__init__()
        self.cosine_loss = CosineLoss(margin=margin, reduction=reduction)
        self.mse_loss = MSELoss(reduction=reduction)

    def forward(self, real, pred, x1, x2, y, margin=0):
        cos_loss = self.cosine_loss(x1, x2, y)
#         print(cos_loss)
        mse_loss = self.mse_loss(real, pred)

        return 0.7 * cos_loss + mse_loss * 0.3, cos_loss, mse_loss

In [94]:
vae = VAE().to(device)
vae_loss = VaeLoss(margin=0, reduction='sum')

In [95]:
# vae.load_state_dict(torch.load('vae.torch', map_location=device))

In [96]:
def get_optimizer(lr, models):
    return torch.optim.SGD([{'params': model.parameters()} for model in models],
                    lr=lr, nesterov=True, momentum=0.9, weight_decay=5e-4)

In [97]:
vae.train(True)
lr = 1e-5 * 3
for epoch in range(10):
    train_loss = 0.
    train_cos_loss = 0.
    train_mse_loss = 0.
    train_recall = 0.
    count = 0
    optimizer = get_optimizer(lr, [vae])
    for X, y in pairs_iterator(df_train, chunksize=40000, normalize=False):
        X = torch.tensor(X, device=device, dtype=torch.float, requires_grad=True)
        y = torch.tensor(y, device=device, dtype=torch.float)

        optimizer.zero_grad()
        
        decoded, thetas, encoding = vae(X)
        
        pred_1, pred_2 = encoding[0::2], encoding[1::2]
        x1, x2 = thetas[0::2], thetas[1::2]
        recall = recall_at_k(pred_1, pred_2, y)
        loss, cos_loss, mse_loss = vae_loss(X, decoded, x1, x2, y)
#         loss = criterion(x1, x2, y)
        loss.backward()
#         print(head.embeddings.grad)
        optimizer.step()
        train_loss += loss.item()
        train_cos_loss += cos_loss.item()
        train_mse_loss += mse_loss.item()
        train_recall += recall.cpu().detach().numpy()
        count += 1
        print('Epoch:', epoch, 
              'Loss:', train_loss // count, 
              'Recall:', train_recall / count, 
              'Cos Loss:', train_cos_loss // count, 
              'MSE Loss:', train_mse_loss // count)
        
        if count % 100:
            torch.save(vae.state_dict(), 'vae.torch')
    lr = max(lr / 10, 1e-10)

NameError: name 'df_train' is not defined

In [57]:
def predict_cluster_with_vae(vae, file, chunksize=100000):
    vae.eval()
    
    with torch.no_grad():
        count = 0
        df = pd.read_csv(file,
                         chunksize=chunksize,
                         header=0,
                         converters={'image': read_string_array})

        result = []

        for chunk in df:
            print(count + len(chunk))
            X, _ = read_chunk(chunk)
            X = np.array(X.tolist(), dtype=np.float)
            X = torch.tensor(X, device=device, dtype=torch.float)

            clusters = vae.encode(X).detach().argmax(axis=1)
            result.append(clusters)

            count += len(chunk)
        return torch.cat(result, 0).cpu().numpy()

In [58]:
preds = predict_cluster_with_vae(vae, train_file, 100000)

100000
200000
300000
343350


In [59]:
clusters = get_buckets(cluster_predictions=preds)
clusters

{120: [0,
  5,
  12,
  16,
  22,
  24,
  44,
  49,
  57,
  64,
  123,
  126,
  128,
  152,
  159,
  166,
  175,
  178,
  179,
  180,
  199,
  206,
  219,
  279,
  284,
  301,
  304,
  323,
  325,
  338,
  339,
  358,
  364,
  387,
  392,
  406,
  413,
  438,
  455,
  456,
  465,
  471,
  476,
  484,
  513,
  524,
  528,
  544,
  557,
  558,
  562,
  569,
  578,
  604,
  614,
  629,
  631,
  637,
  690,
  704,
  716,
  728,
  769,
  774,
  775,
  792,
  795,
  802,
  805,
  817,
  818,
  831,
  841,
  842,
  848,
  858,
  887,
  904,
  906,
  926,
  934,
  976,
  984,
  985,
  987,
  998,
  1027,
  1029,
  1047,
  1051,
  1054,
  1064,
  1076,
  1091,
  1093,
  1110,
  1118,
  1124,
  1131,
  1133,
  1136,
  1141,
  1158,
  1170,
  1244,
  1248,
  1253,
  1254,
  1301,
  1327,
  1329,
  1338,
  1350,
  1360,
  1369,
  1381,
  1391,
  1401,
  1414,
  1417,
  1430,
  1452,
  1491,
  1502,
  1536,
  1545,
  1566,
  1584,
  1589,
  1618,
  1621,
  1636,
  1647,
  1677,
  1691,
  1700,
  175

In [60]:
len(clusters)

60

In [61]:
for i in clusters:
    print(len(clusters[i]))

16456
25384
8731
23188
26579
6460
13569
7938
17050
14745
24744
4657
29019
23649
12467
275
6045
7907
22135
11661
3424
7781
13743
6044
1681
2039
3891
213
115
73
425
75
126
43
329
440
5
43
95
29
2
17
1
5
2
4
5
3
2
1
4
15
2
2
6
2
1
1
1
1


In [ ]:
def find_with_vae(vae, clusters, query, df, treshold, chunksize=100000, device=device):
    net.eval()

    similiarity = torch.nn.CosineSimilarity(dim=1)
    similiarity = similiarity.to(device)
    similiarity.eval()

    with torch.no_grad():
        count = 0

        Q = torch.tensor([query], device=device, dtype=torch.float)
        treshold = torch.tensor(treshold, device=device)
        clusters_hat = vae.encode(Q)
        clusters_sorted = torch.argsort(-clusters_hat).cpu().numpy()[0]
        print(clusters_sorted)

        for cluster_idx in clusters_sorted:
            print(cluster_idx)
            cluster = clusters.get(cluster_idx)
            if not cluster:
                continue
            cluster = np.array(clusters[cluster_idx])

            for indexes in chunk_iterator(cluster, chunksize=chunksize):
                X, y = collect_by_indexes(df, indexes, chunksize)

                X, y = to_tensor(X, y, device)

                sim = similiarity(Q, X)

                idx = torch.argmax(sim)

                if sim[idx] > treshold:
                    return y[idx]
        return None